In [1]:
!pip install "sap-llm-commons[all]"

In [2]:
!pip install ai-api-client-sdk

In [1]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\kylin\AppData\Local\Temp\ipykernel_10688\3646097473.py:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [2]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [3]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

In [4]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [6]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ["TAVILY_API_KEY"] = sk['tavily']

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [7]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

In [8]:
PDF_NAME="ai.pdf"
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()

print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')

There are 86 document(s) in ai.pdf.
There are 117 characters in the first page of your document.


Deploy customized fine-tuned models

In [9]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [10]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

In [11]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [ ]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [12]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [13]:
llm.predict("请打印秋夜将晓出篱门迎凉有感这首诗")

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'秋夜将晓出篱门，迎凉有感\n\n秋夜将晓，我走出篱门，\n迎接着凉爽的秋风。\n月亮渐渐西沉，星星点点闪烁，\n夜空中弥漫着宁静的氛围。\n\n凉风拂过脸颊，带来一丝清凉，\n让我感受到秋天的气息。\n树叶沙沙作响，落下一地黄金，\n仿佛是大自然送给我的礼物。\n\n我静静地站在篱门前，\n凝望着远方的夜景。\n思绪随着风儿飘荡，\n回忆起往昔的点点滴滴。\n\n秋夜的寂静让我沉思，\n回忆中的美好时光涌上心头。\n'

In [14]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="AI.pdf.guide")

j:\python3.12\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [15]:
query = "how can i use the sap ai instead of openai?"
similar_docs = vectorstore.similarity_search(query, 3)
similar_docs

[Document(page_content='use cases.\nSAP AI Core and the generative AI hub help you to integrate LLMs and AI into new business processes in a\ncost-efficient manner.\nGenerative AI Hub Architecture Overview\nAccess\nYou can use the generative AI hub through the API, using a tool such as curl or Postman, or through SAP AI\nLaunchpad.\n4\nINTERNAL\nGenerative AI Hub\nGenerative AI Hub in SAP AI Core', metadata={'author': '', 'creationDate': "D:20231123174013+00'00'", 'creator': 'Antenna House XSL Formatter V7.3 MR4 Linux : 7.3.5.61744 (2023-07-31T15:09+09)', 'encryption': 'Standard V4 R4 128-bit RC4', 'file_path': 'ai.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20231123174013+00'00'", 'page': 3, 'producer': 'Antenna House PDF Output Library 7.3.1867', 'source': 'ai.pdf', 'subject': '', 'title': '', 'total_pages': 86, 'trapped': ''}),
 Document(page_content='6.2 \nConsume Large Language Models Using SAP AI Launchpad. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 62

In [17]:
query = "What is the world's best B2B company?"
query_embedding = embedding.embed_query(query)

In [21]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [16]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict("What is the world's best company to work?")

'This can vary depending on personal preferences, career goals and specific industries. However, as of 2020, according to "Fortune" magazine, the world\'s best company to work for is Salesforce, a cloud-based software company based in San Francisco. They are praised for their strong company culture, benefits, and commitment to philanthropy. Other consistently high-ranking companies include Google, Microsoft, and The Walt Disney Company. The best company for an individual will depend on their own values, goals, and career interests.'

Proxy client - use proxies in parallel

In [17]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from llm_commons.proxy.openai import ChatCompletion

In [18]:
messages=[{"role": "user", "content": "Say this is a test"}]

print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "This is a test.",
        "role": "assistant"
      }
    }
  ],
  "created": 1708225873,
  "id": "chatcmpl-8tRs9nzsShKMzyDSBqnNL45BoDupX",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "detected": false,
          "filtered": false
        },
        "self_harm": {
          "filtered": false

In [19]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

content='This is a test.'


In [20]:
chat = ChatOpenAI(temperature=0,proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
response = chat([ HumanMessage(content="半径为2的圆面积是多少，请直接给出答案以小数显示") ])
print(response)


j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='12.57'


In [21]:
response = chat.predict_messages([
  SystemMessage(content="You are a chatbot that knows nothing about compliance. When you are asked about compliance or audit, you must say 'CX is the best'"),
  HumanMessage(content="What is the best compliance team in SAP")
])
print(response)

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='CX is the best.'


test a search api

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "sap stock"})

[{'url': 'https://www.google.com/finance/quote/SAP:NYSE',
  'content': 'VIX 12.83 -1.76% -0.23 Home SAP • NYSE SAP SE Follow Share $181.18 After Hours: $181.18 (0.00%) 0.00 Closed: Feb 7, 4:09:01 PM GMT-5 · USD · NYSE · Disclaimer search Compare to Oracle Corp...'},
 {'url': 'https://finance.yahoo.com/quote/SAP/',
  'content': 'Crypto Sectors Contact Us U.S. markets close in 44 minutes -3.60 +100.47(+0.26%) -37.28(-.23%) Russell 2000 +36.54(+1.82%) Crude Oil +0.05(+0.07%) Gold 2,033.50 -5.20(-0.26%) SAP SE (SAP) NYSE...'},
 {'url': 'https://www.bloomberg.com/quote/SAP:US',
  'content': 'Stock analysis for SAP SE (SAP:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile.'},
 {'url': 'https://www.marketwatch.com/investing/stock/sap',
  'content': 'Get the latest SAP stock price, news, analysis and charts for the German software giant. See how SAP performs in the cloud, AI and innovation markets, and how it compares to its peers and 

Using llm-commons with fine-tuned models

In [24]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [25]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best B2B company?
Answer: Let's think step by step.

> Finished chain.


'It\'s subjective to determine the "best" B2B company in the world, as it depends on various factors such as the industry, the company\'s impact, its financial performance, customer satisfaction, and innovation. However, some top B2B companies include IBM, Alibaba, Microsoft, and Google Cloud. These companies have a significant impact in their respective industries and are renowned for their innovative solutions and services.'

In [51]:
from langchain.agents import get_all_tool_names
get_all_tool_names()


['requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api']

In [43]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain.tools.tavily_search import TavilySearchResults


# set up the agent
llm = ChatOpenAI(proxy_model_name='gpt-4')
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "AttributeError: module 'autogen' has no attribute 'config_list_from_json'",
)



> Entering new AgentExecutor chain...
The error message indicates that you're trying to access a method or attribute named 'config_list_from_json' from a module named 'autogen', but the module does not have this method or attribute. This could be because the 'config_list_from_json' function doesn't exist in the 'autogen' module, it's not correctly imported, or the 'autogen' module itself might not be correctly installed or imported. Make sure that you have correctly spelled the method or attribute name, and that it exists in the 'autogen' module. Also, check your import statements to confirm that the module is correctly imported. If the function is part of a class in the module, you'll need to create an instance of the class to use the function.

> Finished chain.


"The error message indicates that you're trying to access a method or attribute named 'config_list_from_json' from a module named 'autogen', but the module does not have this method or attribute. This could be because the 'config_list_from_json' function doesn't exist in the 'autogen' module, it's not correctly imported, or the 'autogen' module itself might not be correctly installed or imported. Make sure that you have correctly spelled the method or attribute name, and that it exists in the 'autogen' module. Also, check your import statements to confirm that the module is correctly imported. If the function is part of a class in the module, you'll need to create an instance of the class to use the function."

In [57]:
import pyautogen
print(dir(autogen))

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt4", "gpt-35-turbo"],
    },
)

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

ModuleNotFoundError: No module named 'pyautogen'